In [ ]:
import numpy as np
import cv2
import glob
import matplotlib.pyplot as plt
# HOG & Extraction
from skimage.feature import hog
import time
import matplotlib.image as mpimg
#SVC
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.svm import LinearSVC
import pickle

#### Loading dataset

In [ ]:
cars_dir = [car for car in glob.glob("../dataset/vehicles/*/*.png",recursive=True)]
non_car_dir = [non_car for non_car in glob.glob("../dataset/non-vehicles/*/*.png",recursive=True)]
print("Cars loaded: {} Non cars loaded: {}".format(len(cars_dir),len(non_car_dir)))

#### Loading test Images

In [ ]:
test_images = [plt.imread(i) for i in glob.glob("../test_images/*.jpg")]
plt.imshow(test_images[5])

## HOG

#### Feature Extraction 

In [ ]:
def bin_spatial(img, size=(32, 32)):
    # Use cv2.resize().ravel() to create the feature vector
    features = cv2.resize(img, size).ravel() 
    # Return the feature vector
#     print(features.shape)
    return features

def color_hist(img, nbins=32, bins_range=(0, 256)):
    # Compute the histogram of the color channels separately
    channel1_hist = np.histogram(img[:,:,0], bins=nbins, range=bins_range)
    channel2_hist = np.histogram(img[:,:,1], bins=nbins, range=bins_range)
    channel3_hist = np.histogram(img[:,:,2], bins=nbins, range=bins_range)
    # Concatenate the histograms into a single feature vector
    hist_features = np.concatenate((channel1_hist[0], channel2_hist[0], channel3_hist[0]))
#     Seems like a useless step each channel for all images return same result
#     plt.hist(channel1_hist,bins=32,range=(0,256))
#     plt.hist(channel2_hist,bins=32,range=(0,256))
#     plt.hist(channel3_hist,bins=32,range=(0,256))
#     print(hist_features.shape)
    # Return the individual histograms, bin_centers and feature vector
    return hist_features

# Return HOG features and visualization
# Hog takes only gray images
def get_hog_features(img, orient, pix_per_cell, cell_per_block, 
                        vis=False, feature_vec=True):
    # Call with two outputs if vis==True
    if vis == True:
        features, hog_image = hog(img, orientations=orient, pixels_per_cell=(pix_per_cell, pix_per_cell),
                                  cells_per_block=(cell_per_block, cell_per_block), transform_sqrt=True, 
                                  visualize=vis, feature_vector=feature_vec)
        return features, hog_image
    # Otherwise call with one output
    else:      
        features = hog(img, orientations=orient, pixels_per_cell=(pix_per_cell, pix_per_cell),
                       cells_per_block=(cell_per_block, cell_per_block), transform_sqrt=True, 
                       visualize=vis, feature_vector=feature_vec)
        return features
def convert_rgb_color(img, conv='YCrCb'):
    if conv == 'RGB':
        return np.copy(img)
    if conv == 'YCrCb':
        return cv2.cvtColor(img, cv2.COLOR_RGB2YCrCb)
    if conv == 'HSV':
        return cv2.cvtColor(img, cv2.COLOR_RGB2HSV)
    if conv == 'HLS':
        return cv2.cvtColor(img, cv2.COLOR_RGB2HLS)
    if conv == 'LUV':
        return cv2.cvtColor(img, cv2.COLOR_RGB2LUV)
    if conv == 'YUV':
        return cv2.cvtColor(img, cv2.COLOR_RGB2YUV)
    
# Extract feature wrapper that extracts and combines all features
def extract_features(imgs,params):
    cspace= params['color_space']
    hog_channel=params['hog_channel']
    orient=params['orient']
    pix_per_cell= params['pix_per_cell']
    cell_per_block=params['cell_per_block']
    spatial_size=params['spatial_size']
    hist_bins=params['hist_bins']
    hist_range=params['hist_range']
    # Create a list to append feature vectors to
    features = []
    # Iterate through the list of images
    for file in imgs:
        # Read in each one by one
        image = mpimg.imread(file)
        # apply color conversion if other than 'RGB' 
        feature_image = convert_rgb_color(image,cspace)
        # Call get_hog_features() with vis=False, feature_vec=True
        if hog_channel == 'ALL':
            hog_features = []
            for channel in range(feature_image.shape[2]):
                hog_features.append(get_hog_features(feature_image[:,:,channel], 
                                    orient, pix_per_cell, cell_per_block, 
                                    vis=False, feature_vec=True))
            hog_features = np.ravel(hog_features) #Join all channels hog features into one feature vector        
        else:
            hog_features = get_hog_features(feature_image[:,:,hog_channel], orient, 
                        pix_per_cell, cell_per_block, vis=False, feature_vec=True)
        
        # Apply bin_spatial() to get spatial color features
        spatial_features = bin_spatial(feature_image, size=spatial_size)
        # Apply color_hist() also with a color space option now
        hist_features = color_hist(feature_image, nbins=hist_bins, bins_range=hist_range) # Seems useless (?)
        # Append the new feature vector to the features list
        features.append(np.concatenate((spatial_features, hist_features,hog_features))) # Add all features into one feature vector
        
    # Return list of feature vectors
    return features

In [ ]:
## Parameters
params={}
params['color_space'] ='YCrCb'
params['hog_channel'] ='ALL'
params['orient'] = 9
params['pix_per_cell'] = 8
params['cell_per_block'] = 2
params['spatial_size'] = (32,32)
params['hist_bins'] = 32 
params['hist_range'] = (0,256)

## SVM 

#### Extracting features

In [ ]:
t1=time.time()

cars_features = extract_features(cars_dir,params)
notcars_features = extract_features(non_car_dir,params) 

t2 = time.time()
print(round(t2-t1, 2), 'second to extract features (HOG,spatial and color features).')

#### Data Standardization Step

In [ ]:
#Making one feature vector that starts with cars and ends with non_cars
X = np.vstack((cars_features, notcars_features)).astype(np.float64)
#Computes the mean and standard deviation to be used for later scaling.
X_scaler = StandardScaler().fit(X)  #z = (x - u) / s

# Apply the scaler to X
scaled_X = X_scaler.transform(X) #Perform standardization by centering and scaling.

# scaled_X = StandardScaler().fit_transform(X) #  Does both steps at once
# Labeling feature vector <Cars = 1> <Not Cars = 0>
y = np.hstack((np.ones(len(cars_dir)), np.zeros(len(non_car_dir))))

#### Data Split train & test

In [ ]:
rand_state = np.random.randint(0, 100) # Store state and also generate random test-train splits
X_train, X_test, y_train, y_test = train_test_split(scaled_X, y, test_size=0.2, random_state=rand_state)
print('Number of samples in train set: ', len(X_train))
print('Number of samples in test set: ', len(X_test))

In [ ]:
svc = LinearSVC()
#Training SVC
t1 = time.time()
svc.fit(X_train, y_train)
print(time.time()-t1,'Seconds to train SVC...')
# Find accuracy using test set
print('SVC Accuracy = {:.4}'.format(svc.score(X_test, y_test)))
# Check the prediction time for 100 samples
t=time.time()
n_predict = 100
print('My SVC predicts: ', svc.predict(X_test[0:n_predict]))
print('For these',n_predict, 'labels: ', y_test[0:n_predict])
t2 = time.time()
print(round(t2-t, 5), 'Seconds to predict', n_predict,'labels with SVC')

#### Save to pickle file step

In [ ]:
dist_pickle = {}
dist_pickle["svc"] = svc
dist_pickle["scaler"] = X_scaler
pickle.dump(dist_pickle, open("classifier.p", 'wb') )
print('Classifier saved to a pickle file')

## Car Detection

#### Load classifer

In [ ]:
dist_pickle = {}
dist_pickle = pickle.load( open("classifier.p", "rb" ) )


In [ ]:
## Redefining parameters
params['svc'] = dist_pickle['svc']
params['scaler'] = dist_pickle['scaler']
params['color_space'] ='YCrCb'
params['hog_channel'] ='ALL'
params['orient'] = 9
params['pix_per_cell'] = 8
params['cell_per_block'] = 2
params['cells_per_step'] = 2
params['spatial_size'] = (32,32)
params['hist_bins'] = 32 
params['hist_range'] = (0,256)
params['ystart_ystop_scale'] = [(405, 510, 1),(400, 600, 1.5), (500, 710, 2.5) ] 

In [ ]:
def detect_cars(img,params, h_shift=0, visualisation=False):
     #Parameters:
    svc=params['svc']
    X_scaler=params['scaler']
    orient=params['orient']
    cells_per_step = params['cells_per_step']
    pix_per_cell=params['pix_per_cell']
    cell_per_block=params['cell_per_block']
    spatial_size=params['spatial_size']
    hist_bins=params['hist_bins']
    ystart_ystop_scale = params['ystart_ystop_scale']
    
    #Boxes
    detected_boxes_list = [] # All detected cars in an image
    window_visited_list = [] # All windows in an image
    
    #to make computation efficient by reducing values between 0 to 1
    draw_img = np.copy(img)
    img = img.astype(np.float32)/255
    
    #Searching at different window sizes to get as much detection
    for (ystart, ystop, scale) in ystart_ystop_scale:
        
        #Crop region of interest:
        #Far of the camera should have smaller window scale, closer should have bigger window scale.
        
        search_img = img[ystart:ystop, :, :]
        conv_color_img = cv2.cvtColor(search_img, cv2.COLOR_RGB2YCrCb)
        
        H,W,_ = conv_color_img.shape
        if scale != 1:
            conv_color_img = cv2.resize(conv_color_img, (np.int(W/scale), np.int(H/scale)))
        
        ch1 = conv_color_img[:,:,0]
        ch2 = conv_color_img[:,:,1]
        ch3 = conv_color_img[:,:,2]
        
        # Define number of blocks:
        nxblocks = (ch1.shape[1] // pix_per_cell) - cell_per_block # All possible x blocks
        nyblocks = (ch1.shape[0] // pix_per_cell) - cell_per_block # All possible y blocks
        nfeat_per_block = orient*cell_per_block**2 # Just a measurement of number of features
        
        # Window size is 64x64
        window = 64
        #Number of blocks per window
        nblocks_per_window = (window // pix_per_cell) - cell_per_block # can also mean no of cells / window
        nxsteps = (nxblocks - nblocks_per_window) // cells_per_step
        nysteps = (nyblocks - nblocks_per_window) // cells_per_step
        #nxsteps * nysteps = number of windows I think
        #print("nxsteps:",nxsteps,"nysteps:",nysteps, "number of windows:",nxsteps*(nysteps))
        
        # Compute individual channel HOG features for the entire image
        # This is an optimization much better than getting hog for each single window
        hog1 = get_hog_features(ch1, orient, pix_per_cell, cell_per_block, feature_vec=False)
        hog2 = get_hog_features(ch2, orient, pix_per_cell, cell_per_block, feature_vec=False)
        hog3 = get_hog_features(ch3, orient, pix_per_cell, cell_per_block, feature_vec=False)
        
        # Sliding Window
        box_vis=[]
        for xb in range(nxsteps):
            for yb in range(nysteps):
                ypos = yb*cells_per_step
                xpos = xb*cells_per_step
                # Extract HOG for this patch
                # Hog cells in x , y
                hog_feat1 = hog1[ypos:ypos+nblocks_per_window, xpos:xpos+nblocks_per_window].ravel() 
#                 print(hog1[ypos:ypos+nblocks_per_window, xpos:xpos+nblocks_per_window])
                hog_feat2 = hog2[ypos:ypos+nblocks_per_window, xpos:xpos+nblocks_per_window].ravel() 
                hog_feat3 = hog3[ypos:ypos+nblocks_per_window, xpos:xpos+nblocks_per_window].ravel() 
                hog_features = np.hstack((hog_feat1, hog_feat2, hog_feat3))
                
                # Position of window in pixels in the scaled cropped image
                xleft = xpos*pix_per_cell
                ytop = ypos*pix_per_cell
                
                # Extract the image patch covered by current window
                subimg = cv2.resize(conv_color_img[ytop:ytop+window, xleft:xleft+window], (window,window))
                
                # Get color features
                spatial_features = bin_spatial(subimg, size=spatial_size)
                hist_features = color_hist(subimg, nbins=hist_bins)
                
                # Scale features and make a prediction
                test_features = X_scaler.transform(np.hstack((spatial_features, hist_features, hog_features)).reshape(1, -1))    
                
                # Make prediction based on trained model 
                test_prediction = svc.predict(test_features) # Returns 1 if car is detected 0 if not
                
                if(visualisation): # Draw a box over the real image
                    # To view scale and compare it to car sizes
                    xbox_left = np.int(xleft*scale) 
                    ytop_draw = np.int(ytop*scale)
                    win_draw = np.int(window*scale)
                    # Append Detection Position to list 
                    box_vis.append(((xbox_left, ytop_draw+ystart),(xbox_left+win_draw,ytop_draw+win_draw+ystart)))
                # Detected cars go here
                if test_prediction == 1:
                    xbox_left = np.int(xleft*scale)
                    ytop_draw = np.int(ytop*scale)
                    win_draw = np.int(window*scale)
                    # Append Detection Position to list 
                    detected_boxes_list.append(((xbox_left+h_shift, ytop_draw+ystart),(xbox_left+win_draw+h_shift,ytop_draw+win_draw+ystart)))

        window_visited_list += [box_vis]
    return detected_boxes_list, window_visited_list
        
        
            

    
    

In [ ]:
detection_img = test_images[5]
detected,visited = detect_cars(detection_img,params)